In [ ]:
import requests
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# --- Load MobileNetV2 Model ---
mobilenet_model = MobileNetV2(weights='imagenet')

# --- Function to Get Nutrition Info ---
def get_food_nutrition(food_item):
    api_key = "N7Lwn54DklixpmXQ6N3R3mCi8QabqpVWLjQgfw6C"  # Replace with your USDA API key
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={food_item}&api_key={api_key}"
    try:
        response = requests.get(url).json()
        food_name = response['foods'][0]['description']
        nutrients = response['foods'][0]['foodNutrients']
        top_nutrients = "\n".join([f"{n['nutrientName']}: {n['value']} {n['unitName']}" for n in nutrients[:5]])
        return food_name, top_nutrients
    except:
        return "No data found", "Nutrition details unavailable."

# --- Function to Predict Food from Image ---
def image_food_detector(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        predictions = mobilenet_model.predict(img_array)
        decoded_predictions = decode_predictions(predictions, top=3)[0]

        print("\n📸 Top predictions:")
        for i, pred in enumerate(decoded_predictions):
            print(f"{i+1}. {pred[1]} ({pred[2]*100:.2f}%)")

        best_pred = decoded_predictions[0]
        if best_pred[2] < 0.30:
            print("⚠ Low confidence. Consider uploading a clearer image.")
        return best_pred[1]

    except Exception as e:
        print(f"❌ Error processing image: {e}")
        return "Unknown"

# --- Main Program ---
def run_nutrition_assistant():
    print("\n🔍 Modes available: text / image")
    mode = input("Choose input mode: ").strip().lower()

    if mode == "text":
        food = input("Enter food item: ")
    elif mode == "image":
        image_path = input("Enter image path (e.g., mango.jpg): ")
        food = image_food_detector(image_path)
    else:
        print("❌ Invalid mode")
        return

    food_name, nutrition_info = get_food_nutrition(food)

    print("\n🍎 Food:", food_name)
    print("🧪 Nutrition Info:\n", nutrition_info)

# --- Run the App ---
if __name__ == "__main__":
    run_nutrition_assistant()